In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/pjsofts/learnml/main/data/house-prices-advanced-regression-techniques/train.csv", index_col=0)

In [ ]:
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
numeric_columns = train_data.select_dtypes(include="number").columns
train_data[numeric_columns] = train_data[numeric_columns].fillna(-1)

In [ ]:
character_columns = train_data.select_dtypes(include="object").columns
train_data[character_columns] = train_data[character_columns].fillna("MISSING")

In [ ]:
train_data.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [ ]:
train_data = pd.get_dummies(train_data, columns=character_columns)

In [ ]:
from sklearn.linear_model import LinearRegression , Ridge , Lasso
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor ,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.kernel_ridge import  KernelRidge

In [ ]:
y = train_data.pop('SalePrice')
columns = ['LotFrontage',
 'LotArea',
 'OverallQual',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'FullBath',
 'TotRmsAbvGrd',
 'GarageCars',
 'GarageArea',
 'BsmtQual_Ex']
X = train_data[columns]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val,  y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state = 40)

In [ ]:
integers = ['max_depth','min_samples_leaf',
            'min_samples_split','n_estimators',
            'n_neighbors', 'fit_intercept']
model_search_spaces = {'XGBRegressor()': {
        'learning_rate': (0.01, 0.3),
        'max_depth': (3, 10),
        'subsample': (0.5, 1.0),
        'colsample_bytree': (0.5, 1.0)
    },'ExtraTreesRegressor()': {
        'n_estimators': (100, 1000),
        'max_features': (0.1, 0.9),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 10)
    },
    'RandomForestRegressor()': {
        'n_estimators': (100, 1000),
        'max_features': (0.1, 0.9),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 10)
    },
'KNeighborsRegressor()': {
        'n_neighbors': (1, 10),
        #'weights': ('uniform','distance')
    },
    'BayesianRidge()': {
        'alpha_1': (1e-6, 1),
        'alpha_2': (1e-6, 1),
        'lambda_1': (1e-6, 1),
        'lambda_2': (1e-6, 1),
        'fit_intercept': (0, 1)
    },
    'KernelRidge()': {
        'alpha': (1e-6, 1),
        'gamma': (1e-6, 1),


    },
    'Ridge()': {'alpha':(1e-6,0.9),'tol':(1e-4,0.9)}
                       ,
    'Lasso()': {'alpha':(1e-6,1),'tol':(1e-4,1)}
                       ,
}

In [ ]:
#pip install bayesian_optimization

In [ ]:
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

In [ ]:
def tune_model(model_name, search_space):
    model = eval(model_name)  # Convert string to model object

    def objective_function(**params):
        for param_name , param_value in params.items() :
          if param_name in integers :
            params[param_name] = round(param_value)
        model.set_params(**params)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_val)
        mse = mean_squared_error(y_val, y_pred)
        return -mse  # Negative sign for maximization

    optimization = BayesianOptimization(f=objective_function, pbounds=search_space)
    optimization.maximize(init_points=5, n_iter=5)

    # Retrieve the best hyperparameters
    best_params = optimization.max['params']
    print("Best Hyperparameters for {}: {}".format(model_name, best_params))

    # Train the model with the best hyperparameters
    best_model = eval(model_name)
    for param_name , param_value in best_params.items() :
        if param_name in integers :
          best_params[param_name] = round(param_value)
    best_model.set_params(**best_params)
    best_model.fit(x_train, y_train)

    return best_model

# Perform Bayesian Optimization for each model and train with the best hyperparameters
models_first_layer = []

for model_name, search_space in model_search_spaces.items():
    best_model = tune_model(model_name, search_space)
    models_first_layer.append(best_model)

|   iter    |  target   | colsam... | learni... | max_depth | subsample |
-------------------------------------------------------------------------
| 1         | -9.085e+0 | 0.8375    | 0.2221    | 6.282     | 0.7795    |
| 2         | -1.56e+09 | 0.7085    | 0.2435    | 8.665     | 0.5201    |
| 3         | -8.294e+0 | 0.5648    | 0.06671   | 6.398     | 0.8934    |
| 4         | -1.814e+0 | 0.8574    | 0.01881   | 9.28      | 0.5283    |
| 5         | -3.386e+0 | 0.8325    | 0.01359   | 6.299     | 0.6311    |
| 6         | -9.834e+0 | 0.5328    | 0.1349    | 7.001     | 0.503     |
| 7         | -1.113e+0 | 0.6468    | 0.223     | 6.688     | 0.9249    |
| 8         | -8.388e+0 | 0.5683    | 0.0734    | 8.933     | 0.9772    |
| 9         | -1.058e+0 | 0.9562    | 0.2919    | 3.131     | 0.9935    |
| 10        | -9.284e+0 | 0.6772    | 0.05254   | 7.921     | 0.9795    |
Best Hyperparameters for XGBRegressor(): {'colsample_bytree': 0.564804677902477, 'learning_rate': 0.066706775442

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarnin

| 1         | -1.334e+0 | 0.4624    | 0.4432    | 0.4644    | 0.09827   | 0.02225   |
| 2         | -8.624e+0 | 0.2881    | 0.1372    | 0.5541    | 0.8076    | 0.02511   |
| 3         | -8.624e+0 | 0.6956    | 0.5148    | 0.8763    | 0.7915    | 0.1038    |
| 4         | -8.625e+0 | 0.4209    | 0.784     | 0.9552    | 0.9732    | 0.01637   |
| 5         | -1.334e+0 | 0.5562    | 0.405     | 0.4897    | 0.7628    | 0.9216    |


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


| 6         | -8.625e+0 | 0.3386    | 0.2412    | 1.0       | 1.0       | 1e-06     |


...

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


| 10        | -1.334e+0 | 0.7991    | 1e-06     | 0.3834    | 1.0       | 1e-06     |
Best Hyperparameters for BayesianRidge(): {'alpha_1': 0.6955734860896999, 'alpha_2': 0.5148381036860521, 'fit_intercept': 0.8763443989881677, 'lambda_1': 0.7915360096628686, 'lambda_2': 0.10380400377193771}
|   iter    |  target   |   alpha   |   gamma   |
-------------------------------------------------
| 1         | -8.804e+0 | 0.05838   | 0.5267    |
| 2         | -8.804e+0 | 0.05786   | 0.6789    |
| 3         | -8.804e+0 | 0.2737    | 0.5999    |
| 4         | -8.803e+0 | 0.05218   | 0.4669    |
| 5         | -8.804e+0 | 0.08466   | 0.929     |
| 6         | -8.804e+0 | 0.2043    | 0.01678   |
| 7         | -8.803e+0 | 0.6895    | 0.5116    |
| 8         | -8.804e+0 | 0.05749   | 0.4824    |
| 9         | -8.803e+0 | 0.7166    | 0.7234    |
| 10        | -8.804e+0 | 0.2347    | 0.1106    |
Best Hyperparameters for KernelRidge(): {'alpha': 0.05217716710107305, 'gamma': 0.466892234960342}
|   iter

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.735e+11, tolerance: 7.699e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.577e+11, tolerance: 1.931e+10
  model = cd_fast.enet_coordinate_descent(


| 9         | -8.595e+0 | 0.1639    | 0.002507  |
| 10        | -8.595e+0 | 0.06844   | 0.09002   |
Best Hyperparameters for Lasso(): {'alpha': 0.16390542475729905, 'tol': 0.002507448444836626}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.669e+11, tolerance: 6.931e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.577e+11, tolerance: 1.931e+10
  model = cd_fast.enet_coordinate_descent(


In [ ]:
models_first_layer

[XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.564804677902477, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06670677544243442,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 ExtraTreesRegressor(max_features=0.8381291627613593, min_samples_leaf=3,
                     min_samples_split=3, n_estimators=556),
 RandomForestRegressor(max_features=0.40542808681690046, min_samples_leaf=9,
          

In [ ]:
prediction_first_layer = []
for model in models_first_layer :
  model.fit(x_train,y_train)
  prediction_first_layer.append(model.predict(x_val))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.577e+11, tolerance: 1.931e+10
  model = cd_fast.enet_coordinate_descent(


In [ ]:
import numpy as np

outliers_indices = []
for i , predict in enumerate(prediction_first_layer):
    residuals = y_val - prediction_first_layer[i]
    mean_residuals = np.mean(residuals)
    std_residuals = np.std(residuals)
    outlier_threshold = 3 * std_residuals
    outliers_indices.extend(np.where(np.abs(residuals) > outlier_threshold)[0])
    print(np.where(np.abs(residuals) > outlier_threshold)[0])

outliers_indices = np.unique(outliers_indices)
outliers_indices

[  2  18  51 189 282]
[  2  18  51 189 282]
[  2  18  51 109 189 282]
[  2  18 119 142]
[  2  18 171 282]
[  2  18 105 282]
[  2  18 171 282]
[  2  18 171 282]


array([  2,  18,  51, 105, 109, 119, 142, 171, 189, 282])

In [ ]:
prediction_first_layer

[array([178605.9  , 170725.11 , 293373.97 , 172469.3  , 254136.97 ,
        110734.87 , 170065.23 ,  81846.914, 120681.55 , 216475.94 ,
        113753.53 , 339282.06 , 185544.44 , 101153.94 , 132760.73 ,
        263891.84 , 174402.95 , 271721.6  , 379675.2  , 145132.77 ,
        205667.5  , 183178.53 , 133130.19 , 145049.17 , 190662.75 ,
        130394.89 , 262304.56 , 205610.84 , 143148.33 , 179947.08 ,
        127374.24 , 253891.34 , 145554.72 , 172775.61 , 146626.66 ,
        153752.67 , 155496.62 , 287417.38 , 200740.14 , 302564.34 ,
         70933.1  , 136783.5  , 136071.73 , 168191.44 , 106058.23 ,
         95615.3  , 254905.64 , 278785.56 , 158781.97 , 192711.72 ,
        201085.31 , 295976.2  , 142687.38 , 133573.86 , 191378.88 ,
        225119.3  ,  92241.64 , 217794.81 , 143913.48 ,  87762.445,
        132910.27 , 140074.95 , 221395.14 , 150440.62 , 133539.55 ,
        325075.25 , 277565.12 , 150337.56 , 122112.47 , 205318.56 ,
        192128.33 , 109227.36 , 161017.31 , 1555

In [ ]:
x_second_stage = pd.DataFrame({'Prediction{}'.format(i): pred for i, pred in enumerate(prediction_first_layer)})

In [ ]:
x_second_stage_no_out_lier = x_second_stage.drop(outliers_indices)

In [ ]:
y_val = pd.Series(y_val.values)
y_val_no_out_lier = y_val.drop(outliers_indices)

In [ ]:
x_second_stage_no_out_lier.head()

,Prediction0,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7
0,178605.906250,176878.199915,173241.304194,144555.555556,194832.029584,192126.0,194397.495423,194396.499931
1,170725.109375,175066.748321,166879.177194,189111.111111,179878.860278,178548.0,179756.856812,179757.548943
3,172469.296875,173064.668949,177623.657126,184316.666667,184154.511290,177450.0,184253.893274,184253.747515
4,254136.968750,250004.242464,254539.491677,266405.555556,250826.740997,245755.0,249239.893569,249240.696810
5,110734.867188,109528.369466,108998.695553,102320.333333,83150.392584,75044.0,83638.334585,83638.742220


In [ ]:
x_second_stage.shape , x_second_stage_no_out_lier.shape

((292, 8), (282, 8))

In [ ]:
y_val_no_out_lier.head()

0    190000
1    173000
3    181000
4    284000
5     88000
dtype: int64

In [ ]:
X_train_2,X_val_2,y_train_2,y_val_2 = train_test_split(x_second_stage_no_out_lier,y_val_no_out_lier,test_size=0.2 , random_state=42)

In [ ]:
model_search_space = {'XGBRegressor()':{'learning_rate': (0.01, 0.3),
        'max_depth': (3, 10),
        'subsample': (0.5, 1.0),
        'colsample_bytree': (0.5, 1.0)},'GradientBoostingRegressor()':{'learning_rate':(0,1),'n_estimators':(100,500),'subsample':(0.5,1)},'RandomForestRegressor()':{'n_estimators':(100,500),'max_depth':(5,10),'min_samples_split':(5,10)},'Ridge()':{'alpha':(1e-6,0.9),'tol':(1e-4,0.9)},'Lasso()':{'alpha':(1e-6,1),'tol':(1e-4,1)}}

def tune_model(model_name, search_space):
    model = eval(model_name)  # Convert string to model object

    def objective_function(**params):
        for param_name , param_value in params.items() :
          if param_name in integers :
            params[param_name] = round(param_value)
        model.set_params(**params)
        model.fit(X_train_2, y_train_2)
        y_pred = model.predict(X_val_2)
        mse = mean_squared_error(y_val_2, y_pred)
        return -mse  # Negative sign for maximization

    optimization = BayesianOptimization(f=objective_function, pbounds=search_space)
    optimization.maximize(init_points=5, n_iter=5)

    # Retrieve the best hyperparameters
    best_params = optimization.max['params']
    print("Best Hyperparameters for {}: {}".format(model_name, best_params))

    # Train the model with the best hyperparameters
    best_model = eval(model_name)
    for param_name , param_value in best_params.items() :
        if param_name in integers :
          best_params[param_name] = round(param_value)
    best_model.set_params(**best_params)
    best_model.fit(X_train_2, y_train_2)

    return best_model

models_second_layer = []
for model_name, search_space in model_search_space.items():
    best_model = tune_model(model_name, search_space)
    models_second_layer.append(best_model)

|   iter    |  target   | colsam... | learni... | max_depth | subsample |
-------------------------------------------------------------------------
| 1         | -3.972e+0 | 0.5144    | 0.082     | 3.002     | 0.5033    |
| 2         | -2.054e+0 | 0.5178    | 0.01585   | 3.321     | 0.7342    |
| 3         | -4.281e+0 | 0.5932    | 0.2511    | 8.916     | 0.92      |
| 4         | -4.14e+08 | 0.7242    | 0.04555   | 3.764     | 0.967     |
| 5         | -4.201e+0 | 0.9099    | 0.2191    | 6.805     | 0.7583    |
| 6         | -3.823e+0 | 0.6288    | 0.2274    | 3.283     | 0.7302    |
| 7         | -4.311e+0 | 0.8194    | 0.2591    | 6.836     | 0.8906    |
| 8         | -4.109e+0 | 0.849     | 0.2468    | 6.881     | 0.7959    |
| 9         | -4.693e+0 | 0.8161    | 0.1687    | 8.914     | 0.5228    |
| 10        | -4.458e+0 | 0.9899    | 0.2754    | 4.671     | 0.7355    |
Best Hyperparameters for XGBRegressor(): {'colsample_bytree': 0.628780658625784, 'learning_rate': 0.227378409792

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.755e+10, tolerance: 8.246e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.755e+10, tolerance: 8.246e+07
  model = cd_fast.enet_coordinate_descent(


| 7         | -4.482e+0 | 1.0       | 0.0001    |
| 8         | -4.205e+0 | 0.5923    | 0.6238    |
| 9         | -4.205e+0 | 0.5246    | 0.6419    |
| 10        | -4.205e+0 | 1e-06     | 1.0       |
Best Hyperparameters for Lasso(): {'alpha': 1e-06, 'tol': 1.0}


In [ ]:
models_second_layer

[XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.628780658625784, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.22737840979220755,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 GradientBoostingRegressor(learning_rate=0.32271769445385534, n_estimators=143,
                           subsample=0.9895781653188431),
 RandomForestRegressor(max_depth=8, min_samples_split=5, n_estimators=332),
 Ridge(a

In [ ]:
prediction_second_layer = []
for model in models_second_layer :
  model.fit(X_train_2,y_train_2)
  prediction_second_layer.append(model.predict(X_val_2))

In [ ]:
prediction_second_layer

[array([ 79410.95 , 131874.62 , 144099.31 , 149605.27 , 280983.   ,
        117171.2  , 153966.23 , 120251.64 , 291212.34 , 108325.695,
        202148.47 , 233420.17 , 159174.16 , 260276.4  , 110860.25 ,
        284297.7  , 114742.71 , 221186.94 , 158719.4  , 148323.5  ,
        156488.52 , 244612.97 , 111777.29 , 180838.77 , 178956.62 ,
        149972.98 , 166098.52 , 129826.25 , 129664.44 ,  92138.516,
        148557.73 , 199873.02 , 160308.52 , 143880.84 , 125977.39 ,
        258794.   , 154349.44 , 172088.56 , 149412.53 , 209250.64 ,
        271191.34 , 114896.85 , 198242.38 , 198820.61 , 152842.16 ,
        188286.02 , 135711.16 , 318192.97 , 345734.88 , 178876.23 ,
         92559.43 , 243169.4  , 190915.5  , 115659.266, 156809.8  ,
        199023.   , 128570.85 ], dtype=float32),
 array([ 90176.8602196 , 131820.99730985, 136569.84341255, 135762.23042633,
        273331.61814221, 120874.32224462, 154103.47553157, 109693.36130352,
        282016.04568298, 113516.73853599, 188458.93

In [ ]:
x_third_stage = pd.DataFrame({'Prediction{}'.format(i): pred for i, pred in enumerate(prediction_second_layer)})

In [ ]:
x_third_stage.head()

,Prediction0,Prediction1,Prediction2,Prediction3,Prediction4
0,79410.953125,90176.860220,88090.335482,75729.651271,82605.013534
1,131874.625000,131820.997310,132377.241489,132276.359440,131586.668450
2,144099.312500,136569.843413,143443.903845,142893.768791,140670.308679
3,149605.265625,135762.230426,143508.973082,147417.585969,137655.802537
4,280983.000000,273331.618142,267266.797892,267379.198713,264469.678843


In [ ]:
y_val_2.head()

95      52500
96     147000
187    131500
130    150000
270    275000
dtype: int64

In [ ]:
meta_regressor = LinearRegression()
meta_regressor.fit(x_third_stage, y_val_2)

LinearRegression()

In [ ]:
prediction_third_layer = meta_regressor.predict(x_third_stage)
prediction_third_layer = pd.DataFrame({'Prediction_third_layer':prediction_third_layer})
prediction_third_layer.head()

,Prediction_third_layer
0,72793.900088
1,124987.263441
2,133790.950954
3,132820.092035
4,281145.916496


In [ ]:
from sklearn import metrics
y_pred = meta_regressor.predict(x_third_stage)
print('Mean squared error: %.2f'
% metrics.mean_squared_error(y_val_2, y_pred))


# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination (R^2): %.2f'
% metrics.r2_score(y_val_2, y_pred))


# The mean absolute error
print('Mean absolute error: %.2f'
% metrics.mean_absolute_error(y_val_2, y_pred))


# The root mean squared error
print('Root Mean Squared Error: %.2f'
% np.sqrt(metrics.mean_squared_error(y_val_2, y_pred)))


Mean squared error: 307235682.48
Coefficient of determination (R^2): 0.93
Mean absolute error: 13578.92
Root Mean Squared Error: 17528.14


In [ ]:
test_data = pd.read_csv("https://raw.githubusercontent.com/pjsofts/learnml/main/data/house-prices-advanced-regression-techniques/test.csv", index_col=0)


In [ ]:
test_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
numeric_columns = test_data.select_dtypes(include="number").columns
test_data[numeric_columns] = test_data[numeric_columns].fillna(-1)


character_columns = test_data.select_dtypes(include="object").columns
test_data[character_columns] = test_data[character_columns].fillna("MISSING")

test_data = pd.get_dummies(test_data, columns=character_columns)
X_test = test_data[columns]

In [ ]:
prediction_test_first_layer = []
for model in models_first_layer :
  prediction_test_first_layer.append(model.predict(X_test))

In [ ]:
X_test.shape

(1459, 15)

In [ ]:
prediction_test_first_layer

[array([129338.625, 164621.86 , 179971.   , ..., 181139.98 , 122420.68 ,
        228672.08 ], dtype=float32),
 array([124184.07726961, 158812.42975334, 173118.95325739, ...,
        153793.40460774, 110262.80032235, 238726.59177444]),
 array([128282.00434558, 159849.12990632, 178964.48605797, ...,
        158204.14944964, 120586.17724349, 237696.6672733 ]),
 array([146833.33333333, 177811.11111111, 195200.        , ...,
        204774.44444444, 115355.55555556, 230482.22222222]),
 array([111899.86261657, 166479.04768117, 174261.46558776, ...,
        156925.358989  , 111457.08055624, 240272.8664596 ]),
 array([109556., 165444., 167791., ..., 154874.,  94862., 237708.]),
 array([110352.02782442, 165539.58186104, 174296.78198752, ...,
        155912.79728463, 111089.73970052, 239640.48402061]),
 array([110354.35320348, 165540.6701085 , 174296.08055535, ...,
        155913.72314115, 111091.12487998, 239640.47128907])]

In [ ]:
x_test_second_stage = pd.DataFrame({'Prediction{}'.format(i): pred for i, pred in enumerate(prediction_test_first_layer)})
x_test_second_stage.shape

(1459, 8)

In [ ]:
x_test_second_stage.head()

,Prediction0,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7
0,129338.625000,124184.077270,128282.004346,146833.333333,111899.862617,109556.0,110352.027824,110354.353203
1,164621.859375,158812.429753,159849.129906,177811.111111,166479.047681,165444.0,165539.581861,165540.670108
2,179971.000000,173118.953257,178964.486058,195200.000000,174261.465588,167791.0,174296.781988,174296.080555
3,179072.703125,186124.961226,183737.206525,189100.000000,188428.489082,182340.0,188057.173287,188056.983844
4,190205.734375,198788.328175,204293.248168,166264.444444,208853.075887,212158.5,208072.409483,208071.950743


In [ ]:
prediction_test_second_layer = []
for model in models_second_layer :
  prediction_test_second_layer.append(model.predict(x_test_second_stage))

In [ ]:
prediction_test_second_layer

[array([136119.38, 159174.16, 165636.6 , ..., 158452.52, 110322.6 ,
        253184.8 ], dtype=float32),
 array([130792.22562632, 154990.46608042, 161322.5356488 , ...,
        140904.18951202, 101611.93401676, 245792.70762638]),
 array([129867.66296073, 160181.05990232, 174571.63456161, ...,
        158846.02784019, 104601.08735262, 247399.63019758]),
 array([131483.6615928 , 165286.80583428, 174891.02211387, ...,
        168212.41765768, 114922.87553226, 233034.90674632]),
 array([131490.89842124, 164428.28556979, 179128.37542273, ...,
        179917.0315811 , 123570.48413849, 225175.1745501 ])]

In [ ]:
x_test_third_stage = pd.DataFrame({'Prediction{}'.format(i): pred for i , pred in enumerate(prediction_test_second_layer)})

In [ ]:
x_test_third_stage

,Prediction0,Prediction1,Prediction2,Prediction3,Prediction4
0,136119.375000,130792.225626,129867.662961,131483.661593,131490.898421
1,159174.156250,154990.466080,160181.059902,165286.805834,164428.285570
2,165636.593750,161322.535649,174571.634562,174891.022114,179128.375423
3,176316.031250,167131.317882,174490.292543,182087.815204,178426.676842
4,213747.062500,195544.135360,186951.813199,192588.383574,187575.726680
...,...,...,...,...,...
1454,86730.390625,89982.593283,90816.298064,81572.379793,81453.270915
1455,94831.984375,89858.873008,94856.509619,87732.906929,89180.619737
1456,158452.515625,140904.189512,158846.027840,168212.417658,179917.031581
1457,110322.601562,101611.934017,104601.087353,114922.875532,123570.484138


In [ ]:
y_prediction_test = meta_regressor.predict(x_test_third_stage)

In [ ]:
predictions = pd.DataFrame({"SalePrice":y_prediction_test}, index=test_data.index)
predictions.to_csv("predictions_3layer.csv")

In [ ]:
y_prediction_test

array([131527.60496908, 155019.73022033, 162802.69208356, ...,
       167422.30856655, 118224.83999334, 235718.767535  ])

In [ ]:
from google.colab import files
files.download("predictions_3layer.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>